# Display a Summary of All Activities for a Given Course

In [ ]:
from IPython.display import display, Markdown
import snakemd

import fitfile
from garmindb import GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Attributes, ActivitiesDb, Activities, ActivityLaps, ActivityRecords, StepsActivities

from jupyter_funcs import format_number


doc = snakemd.new_doc()

course_id = input('Enter the id of a course to summarize')

doc.add_heading(f"Analysis for Course {course_id}", 2)

gc_config = GarminConnectConfigManager()
db_params_dict = gc_config.get_db_params()

garmin_db = GarminDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = unit_strings[fitfile.units.UnitTypes.distance_long]

activity_db = ActivitiesDb(db_params_dict)

activities = Activities.get_by_course_id(activity_db, course_id)

activities_count = len(activities)
fastest_activity = Activities.get_fastest_by_course_id(activity_db, course_id)
slowest_activity = Activities.get_slowest_by_course_id(activity_db, course_id)

doc.add_paragraph(f'{activities_count} activities using this course')

def __activity_data(activity, title):
    if activity.is_steps_activity():
        steps_activity = StepsActivities.get(activity_db, activity.activity_id)
        return [title, activity.start_time, activity.activity_id, activity.name, format_number(activity.distance), activity.elapsed_time, steps_activity.avg_pace, format_number(activity.avg_speed)]
    return [title, activity.start_time, activity.activity_id, activity.name, format_number(activity.distance), activity.elapsed_time, '', format_number(activity.avg_speed)]

doc.add_table(
    ['Type', 'On', 'Id', 'Name', f'Distance ({unit_strings[fitfile.units.UnitTypes.distance_long]})', 'Time', f'Pace ({unit_strings[fitfile.units.UnitTypes.pace]})', f'Speed ({unit_strings[fitfile.units.UnitTypes.speed]})'],
    [
        __activity_data(activities[0], "First"),
        __activity_data(activities[-1], "Latest"),
        __activity_data(fastest_activity, "Fastest"),
        __activity_data(slowest_activity, "Slowest")
    ])

display(Markdown(str(doc)))